In [1]:
import cv2 as cv
import numpy as np
import sys

In [2]:
from datasets import load_dataset

dataset = load_dataset("wider_face")

Found cached dataset wider_face (C:/Users/Asus/.cache/huggingface/datasets/wider_face/default/1.0.0/b87ac8d8d65665ec6e3c2a5c6ec08d6fddb1b0f2d7f2dc3b5dcecdaf12adf22f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def pimToCV(pim):
    nimg = np.array(pim)
    img = cv.cvtColor(nimg, cv.COLOR_RGB2BGR)
    return img

In [4]:
def parseDataset(dataset, singleFace, pose, size):
    
    output = []
    # for i in range(1000):
    for i in range(len(dataset)):
        try:
            if(singleFace):
                if len(dataset[i]["faces"]["bbox"]) == 1:
                    output.append(dataset[i])
            else:
                output.append(dataset[i])
            
            
        except:
            print("An exception occurred at index: ", i)
            
    if(pose == "easy"): 
        deletedCount = 0;
        for i in range(len(output)):
            
            cur = output[i-deletedCount]
            
            for j in range(len(cur["faces"]["pose"])):
                if cur["faces"]["pose"][j] == 1:
                    output.pop(i-deletedCount);
                    deletedCount += 1
                    break;
    
                
    size_threshold = 0
    if(size == "small"):
        size_threshold = 10
    if(size == "medium"):
        size_threshold = 50
    if(size == "large"):
        size_threshold = 300
    
    deletedCount = 0;
    for i in range(len(output)):
            
        cur = output[i-deletedCount]
            
        for j in range(len(cur["faces"]["bbox"])):
            if cur["faces"]["bbox"][j][3] < size_threshold:
                output.pop(i-deletedCount);
                deletedCount += 1
                break;
            if cur["faces"]["occlusion"][j] != 0:
                output.pop(i-deletedCount);
                deletedCount += 1
                break;


            
    return output


In [20]:
parsedDataset = parseDataset(dataset["train"], True, "easy", "large")
print(len(parsedDataset))

An exception occurred at index:  581
An exception occurred at index:  582
An exception occurred at index:  586
An exception occurred at index:  590
An exception occurred at index:  600
An exception occurred at index:  604
An exception occurred at index:  605
An exception occurred at index:  607
An exception occurred at index:  610
An exception occurred at index:  613
An exception occurred at index:  614
An exception occurred at index:  615
An exception occurred at index:  624
An exception occurred at index:  625
An exception occurred at index:  627
An exception occurred at index:  631
An exception occurred at index:  632
An exception occurred at index:  636
An exception occurred at index:  638
An exception occurred at index:  639
An exception occurred at index:  642
An exception occurred at index:  643
An exception occurred at index:  645
An exception occurred at index:  650
An exception occurred at index:  660
An exception occurred at index:  664
An exception occurred at index:  667
A

In [14]:
multiFace = parseDataset(dataset["train"], False, "hard", "small")


An exception occurred at index:  581
An exception occurred at index:  582
An exception occurred at index:  586
An exception occurred at index:  590
An exception occurred at index:  600
An exception occurred at index:  604
An exception occurred at index:  605
An exception occurred at index:  607
An exception occurred at index:  610
An exception occurred at index:  613
An exception occurred at index:  614
An exception occurred at index:  615
An exception occurred at index:  624
An exception occurred at index:  625
An exception occurred at index:  627
An exception occurred at index:  631
An exception occurred at index:  632
An exception occurred at index:  636
An exception occurred at index:  638
An exception occurred at index:  639
An exception occurred at index:  642
An exception occurred at index:  643
An exception occurred at index:  645
An exception occurred at index:  650
An exception occurred at index:  660
An exception occurred at index:  664
An exception occurred at index:  667
A

In [15]:
len(multiFace)

7388

In [8]:
sdf

NameError: name 'sdf' is not defined

In [6]:
def findFaceCoordsThreshYCBCR(frame):
     # convert frame to YCrCb color space
    ycbcr = cv.cvtColor(frame, cv.COLOR_BGR2YCrCb)
    
    # define bounds for skin color in YCrCb color space
    lower_skin = np.array([0, 135, 85])
    upper_skin = np.array([255, 180, 135])

    # threshold the image using the bounds above
    thresh = cv.inRange(ycbcr, lower_skin, upper_skin)
    
    # remove noise using morphological operations
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (15,15))
    mask = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    
    # find contours in the mask
    contours, hierarchy = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    # set minimum and maximum width and height to help get rid of false positives
    min_face_size = 100
    max_face_size = 500
    

    coords = [];
    # loop over all contours
    for cnt in contours:
        # get bounding rectangles for contours and compute their aspect ratio 
        x,y,w,h = cv.boundingRect(cnt)
        aspect_ratio = float(w)/h
        
        # if aspect ratio is in a typical range from human faces and rectangle is in the right size range, draw rectangle
        if min_face_size < w < max_face_size and min_face_size < h < max_face_size and 0.5 <= aspect_ratio <= 1.8:
                # cv.rectangle(mask, (x,y), (x+w,y+h), (250,250,250), thickness=2)
                coords.append((x,y,w,h))
    return coords
        


In [7]:
def findFaceCoordsHaar(frame, draw=False):
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

    # convert frame to greyscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
    # detect faces and return their coordinates
    face_coordinates = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(10,10))
   
    if(draw == True):
        for(x,y,w,h) in face_coordinates:
            cv.rectangle(frame, (x,y), (x+w,y+h), (250,250,250), thickness=2)

    return face_coordinates;
    

In [8]:
def findFaceCoordsHaarWithEyes(frame, draw=False):
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
    eye_cascade = cv.CascadeClassifier('haarcascade_eye.xml')

    output = [];
    # convert frame to greyscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
    # detect faces and return their coordinates
    face_coordinates = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(10,10))
    #if no face coordinates are found, return None
    
    for(x,y,w,h) in face_coordinates:
        
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # detect eyes and return their coordinates
        eye_coordinates = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.2, minNeighbors=5, minSize=(10,10))

        if(len(eye_coordinates) > 0):
            output.append((x,y,w,h));
            if(draw):
                cv.rectangle(frame, (x,y), (x+w,y+h), (250,250,250), thickness=2)


        if(draw):
            # create a rectangle around each detected eye
            for(ex,ey,ew,eh) in eye_coordinates:
                cv.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (0,250,0), thickness=2)


    return output;
    

In [9]:
def calculate_iou(box1, box2):
    # Extract coordinates of the bounding boxes
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate the coordinates of the intersection area
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    
    # If there's no intersection, return 0
    if x_right <= x_left or y_bottom <= y_top:
        return 0.0
    
    # Calculate the area of intersection
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    # Calculate the area of each bounding box
    box1_area = w1 * h1
    box2_area = w2 * h2
    
    # Calculate the Union area
    union_area = box1_area + box2_area - intersection_area
    
    # Calculate the IoU
    iou = intersection_area / union_area
    
    return iou

In [10]:
def calculate_precision_recall(predictions, ground_truth, iou_threshold=0.5):
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for pred_box in predictions:
        pred_matched = False

        for gt_box in ground_truth:
            iou = calculate_iou(pred_box, gt_box)

            if iou >= iou_threshold:
                true_positives += 1
                pred_matched = True
                break

        if not pred_matched:
            false_positives += 1
   

    if( true_positives + false_positives == 0 or true_positives + false_negatives == 0):
        return 0, 0, 0
   
    false_negatives = len(ground_truth) - true_positives
    
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)
    f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score
   
    

    

In [11]:
def resizeImage(image):
    # Get the original image dimensions
    height, width = image.shape[:2]

    # Calculate the new dimensions for the resized image
    new_width = int(width / 2)
    new_height = int(height / 2)

    # Rescale the image while preserving aspect ratio
    if width > height:
        resized_width = new_width
        resized_height = int(height * (new_width / width))
    else:
        resized_height = new_height
        resized_width = int(width * (new_height / height))

    # Resize the image
    resized_image = cv.resize(image, (resized_width, resized_height))
    return resized_image

In [17]:
len(parsedDataset)
dataset = parsedDataset

In [12]:
def showPredictions(dataset) :
    for index in range( len(dataset)):
       
        image = dataset[index]['image']
        img = pimToCV(image)

        if img is None:
            sys.exit("Could not read the image.")

        coords = findFaceCoordsHaarWithEyes(img, draw=True)
        
        for coord in coords:
            x,y,w,h = coord
            iou = calculate_iou(coord, dataset[index]['faces']['bbox'][0]);
            img = cv.putText(img, str(round(iou, 2)), (x,y), cv.FONT_HERSHEY_SIMPLEX, 
                    2, (0, 0, 255), 3, cv.LINE_AA)

        real_coords = dataset[index]['faces']['bbox']
        
        print(dataset[index]['faces'])
        for coord in real_coords:
            x,y,w,h = np.asarray(coord, dtype = 'int')
            print(x,y,w,h)
            cv.rectangle(img, (x,y), (x+w,y+h), (0,0,250), thickness=2)   

        
        cv.imshow("Display window", resizeImage(img))
        key = cv.waitKey(0)

        if key == ord('q'):
            cv.destroyAllWindows()
            break;

In [16]:
showPredictions(multiFace)

{'bbox': [[449.0, 330.0, 122.0, 149.0]], 'blur': [0], 'expression': [0], 'illumination': [0], 'occlusion': [0], 'pose': [0], 'invalid': [False]}
449 330 122 149
{'bbox': [[361.0, 98.0, 263.0, 339.0]], 'blur': [0], 'expression': [0], 'illumination': [0], 'occlusion': [0], 'pose': [0], 'invalid': [False]}
361 98 263 339
{'bbox': [[583.0, 379.0, 57.0, 62.0], [676.0, 299.0, 45.0, 60.0], [616.0, 275.0, 39.0, 41.0], [617.0, 224.0, 37.0, 37.0], [453.0, 294.0, 44.0, 53.0], [361.0, 281.0, 33.0, 43.0], [374.0, 380.0, 50.0, 55.0], [250.0, 414.0, 42.0, 53.0], [485.0, 419.0, 48.0, 65.0]], 'blur': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'expression': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'illumination': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'occlusion': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'pose': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'invalid': [False, False, False, False, False, False, False, False, False]}
583 379 57 62
676 299 45 60
616 275 39 41
617 224 37 37
453 294 44 53
361 281 33 43
374 380 50 55
250 414 42 53
485 419 48 65
{'bbox': [[

In [17]:
def calc_average_metrics(dataset, findFaceFunc, iou_threshold=0.5, end = len(dataset)):
    precision_sum = 0
    recall_sum = 0
    f1_score_sum = 0

    # end = len(one_face_dataset);
    # for index in range(len(one_face_dataset)):
    for index in range(end):
        # if(index % 500 == 0):
        #     print(index)
        image = dataset[index]['image']
        img = pimToCV(image)
        # img = cv.imread("photo.jpg")
        if img is None:
            sys.exit("Could not read the image.")

        coords = findFaceFunc(img)
        # coords = findFaceCoordsThreshYCBCR(img)
      
        real_coords = dataset[index]['faces']['bbox']
        
        # print("results = ", calculate_precision_recall(coords, real_coords))
        precision, recall, f1_score = calculate_precision_recall(coords, real_coords, iou_threshold)
        precision_sum += precision
        recall_sum += recall
        f1_score_sum += f1_score
    
    precision_avg = precision_sum / end
    recall_avg = recall_sum / end
    f1_score_avg = f1_score_sum / end

    return precision_avg, recall_avg, f1_score_avg

In [20]:
print(calc_average_metrics(dataset, findFaceFunc=findFaceCoordsHaarWithEyes))

0
100
200
300
400
500
600
700
800
900
(0.6665, 0.671, 0.6680000000000001)


In [21]:
print(calc_average_metrics(dataset, findFaceFunc=findFaceCoordsHaar))

0
100
200
300
400
500
600
700
800
900
(0.6761761904761905, 0.736, 0.6950499999999996)


In [28]:
print(calc_average_metrics(dataset, findFaceFunc=findFaceCoordsHaar, iou_threshold=0.3))

0
500
1000
(0.6993093127662026, 0.765511684125705, 0.7202927746441029)


In [24]:
print(calc_average_metrics(dataset, findFaceFunc=findFaceCoordsThreshYCBCR))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
(0.045280303902382885, 0.0749395648670427, 0.05265147154752309)


In [31]:
hardDataset = parseDataset(dataset["train"], True, "hard", "small")
print(len(hardDataset))


An exception occurred at index:  581
An exception occurred at index:  582
An exception occurred at index:  586
An exception occurred at index:  590
An exception occurred at index:  600
An exception occurred at index:  604
An exception occurred at index:  605
An exception occurred at index:  607
An exception occurred at index:  610
An exception occurred at index:  613
An exception occurred at index:  614
An exception occurred at index:  615
An exception occurred at index:  624
An exception occurred at index:  625
An exception occurred at index:  627
An exception occurred at index:  631
An exception occurred at index:  632
An exception occurred at index:  636
An exception occurred at index:  638
An exception occurred at index:  639
An exception occurred at index:  642
An exception occurred at index:  643
An exception occurred at index:  645
An exception occurred at index:  650
An exception occurred at index:  660
An exception occurred at index:  664
An exception occurred at index:  667
A

In [21]:
print("Easy dataset : ")
print("simple haar : " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsHaar, end=1000)))
print("simple haar with eyes : " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsHaarWithEyes, end=1000)))
print("Threshold based : " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsThreshYCBCR, end=1000)))

Easy dataset : 
simple haar : (0.6761761904761905, 0.736, 0.6950499999999996)
simple haar with eyes : (0.6665, 0.671, 0.6680000000000001)
Threshold based : (0.04699285714285715, 0.078, 0.054707142857142836)


In [35]:
print("simple haar : " + str(calc_average_metrics(hardDataset, findFaceFunc=findFaceCoordsHaar, end=1000)))
print("simple haar with eyes : " + str(calc_average_metrics(hardDataset, findFaceFunc=findFaceCoordsHaarWithEyes, end=1000)))
print("Threshold based : " + str(calc_average_metrics(hardDataset, findFaceFunc=findFaceCoordsThreshYCBCR, end=1000)))

simple haar : (0.5563465608465608, 0.624, 0.576907142857143)
simple haar with eyes : (0.5273333333333332, 0.531, 0.5285)
Threshold based : (0.05642777777777781, 0.093, 0.06567142857142855)


In [18]:
print("simple haar : " + str(calc_average_metrics(multiFace, findFaceFunc=findFaceCoordsHaar, end=1000)))
print("simple haar with eyes : " + str(calc_average_metrics(multiFace, findFaceFunc=findFaceCoordsHaarWithEyes, end=1000)))
print("Threshold based : " + str(calc_average_metrics(multiFace, findFaceFunc=findFaceCoordsThreshYCBCR, end=1000)))

simple haar : (0.6327200747348204, 0.5624392169829087, 0.5666846894215589)
simple haar with eyes : (0.4365833333333334, 0.33947420861302446, 0.3632223668341493)
Threshold based : (0.060834199134199136, 0.06675674603174603, 0.05839540786011371)


In [22]:
print("testing iou thresholds")
print("0.1: " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsHaar, iou_threshold=0.1, end=1000)))
print("0.3 : " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsHaar, iou_threshold=0.3, end=1000)))
print("0.5 : " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsHaar, iou_threshold=0.5, end=1000)))
print("0.7 : " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsHaar, iou_threshold=0.7, end=1000)))
print("0.9 : " + str(calc_average_metrics(parsedDataset, findFaceFunc=findFaceCoordsHaar, iou_threshold=0.9, end=1000)))

NameError: name 'sdfa' is not defined

In [ ]:
dataset['train'][0]['faces']['bbox'][0]

[449.0, 330.0, 122.0, 149.0]

In [ ]:
import cv2
import numpy as np

# capture video from web camera
cap = cv2.VideoCapture(0)

# use loop to capture frame by frame
while(True):

    ret_val, frame = cap.read()
    
    coords = findFaceCoordsHaar(frame);

    # draw rectangle around face
    for (x,y,w,h) in coords:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (250,250,250), thickness=2)
        
    # display the frame with detected faces
    cv2.imshow('Face recognition', frame)
    
    # wait for q key to be pressed to end the face detection screen
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    
# release capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2 as cv
import sys

img = cv.imread("photo.jpg")
# img = cv.imread(cv.samples.findFile("photo2.png"))
if img is None:
    sys.exit("Could not read the image.")
cv.imshow("Display window", img)
k = cv.waitKey(0)
if k == ord("s"):
    cv.imwrite("starry_night.png", img)

In [ ]:
img.shape

(757, 1024, 3)

In [ ]:
import cv2 as cv
nimg = np.array(image)
ocvim = cv.cvtColor(nimg, cv.COLOR_RGB2BGR)
cv.imshow("Display window", ocvim)